## Import packages and set options

In [36]:
# import necessary packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm


In [37]:
# set display options
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
pd.set_option("display.max_rows", 150)


## Data input and cleansing

In [38]:
df = pd.read_csv('./stock-market-data.csv')
df[['symbol', 'industry']] = df[['symbol', 'industry']].apply(
    lambda x: x.astype("string"))
df.head()


,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,index_w50,index_w300,index_w500,industry
0,600000.SH,20120104,8.49,8.54,8.56,8.39,8.41,34201379,290229551.00,6.66,125500555680.00,0.05,0.02,0.00,BANKS
1,600000.SH,20120105,8.41,8.47,8.82,8.47,8.65,132116203,1144753023.00,6.66,129082022192.00,0.05,0.02,0.00,BANKS
2,600000.SH,20120106,8.65,8.63,8.78,8.62,8.71,61778687,537043761.00,6.66,129977388820.00,0.05,0.02,0.00,BANKS
3,600000.SH,20120109,8.71,8.72,8.99,8.68,8.95,80136249,711429611.00,6.66,133558855331.00,0.05,0.02,0.00,BANKS
4,600000.SH,20120110,8.95,8.95,9.10,8.88,9.07,72004632,647206633.00,6.66,135349588587.00,0.05,0.02,0.00,BANKS


## Answers

### 1. 哪些股票的代码中包含"8"这个数字？

In [39]:
# use list comprehension
[s for s in set(df['symbol']) if s.find('8') >= 0][:6]


['600387.SH', '000856.SZ', '000089.SZ', '600808.SH', '601158.SH', '601558.SH']

### 2. 每天上涨和下跌的股票各有多少？

In [40]:
# use np.select
conditions = [
    df['close'] > df['pre_close'],
    df['close'] < df['pre_close']]

results = ['UP', 'DOWN']

df['updown'] = np.select(conditions, results, default='STEADY')
df.groupby(['date', 'updown']).agg(
    num=('symbol', 'nunique')).reset_index().head()


,date,updown,num
0,20120104,DOWN,2007
1,20120104,STEADY,122
2,20120104,UP,191
3,20120105,DOWN,2071
4,20120105,STEADY,117


### 3. 每天每个交易所上涨、下跌的股票各有多少？

In [41]:
# based on question 2

df['exchange'] = df['symbol'].str[-2:]
df.groupby(['date', 'exchange', 'updown']).agg(
    num=('symbol', 'nunique')).reset_index().head()


,date,exchange,updown,num
0,20120104,SH,DOWN,794
1,20120104,SH,STEADY,42
2,20120104,SH,UP,85
3,20120104,SZ,DOWN,1213
4,20120104,SZ,STEADY,80


### 4. 沪深300成分股中，每天上涨、下跌的股票各有多少？

In [42]:
# based on question 2
df.query("index_w300 > 0").groupby(['date', 'updown']).agg(
    num=('symbol', 'nunique')).reset_index().head()


,date,updown,num
0,20120104,DOWN,275
1,20120104,STEADY,5
2,20120104,UP,20
3,20120105,DOWN,242
4,20120105,STEADY,8


### 5. 每天每个行业各有多少只股票？

In [43]:
# based on question 2
df.groupby(['date', 'industry']).agg(
    num=('symbol', 'nunique')).reset_index().head()


,date,industry,num
0,20120104,AERODEF,10
1,20120104,AIRLINE,12
2,20120104,AUTO,85
3,20120104,BANKS,16
4,20120104,BEV,30


### 6. 股票数最大的行业和总成交额最大的行业是否总是同一个行业？

In [44]:
# get # of stocks and $ of amount by industry
df1 = df.groupby(['date', 'industry']).agg(num_total=(
    'symbol', 'nunique'), amount_total=('amount', 'sum')).reset_index()
df1.head()


,date,industry,num_total,amount_total
0,20120104,AERODEF,10,493331236.30
1,20120104,AIRLINE,12,359576492.30
2,20120104,AUTO,85,2299263904.60
3,20120104,BANKS,16,3612012715.00
4,20120104,BEV,30,2946962799.60


In [45]:
df1[['max_num', 'max_amount']] = df1.groupby(
    'date')[['num_total', 'amount_total']].transform('max')
df1.query('num_total == max_num and amount_total == max_amount').head()


,date,industry,num_total,amount_total,max_num,max_amount
15,20120104,HDWRSEMI,224,6878138136.80,224,6878138136.80
79,20120106,HDWRSEMI,224,5979258564.90,224,5979258564.90
751,20120213,HDWRSEMI,227,14602692451.80,227,14602692451.80
847,20120216,HDWRSEMI,227,15064632475.20,227,15064632475.20
879,20120217,HDWRSEMI,229,11451347414.00,229,11451347414.00


### 7. 每天涨幅超过5%、跌幅超过5%的股票各有多少

In [46]:
# similar to question 2
# use np.select

df['change'] = df['close'] - df['pre_close']
df['change%'] = df['change']/df['pre_close']
conditions = [
    df['change%'] > 0.05,
    df['change%']  < -0.05]

results = ['UP5%', 'DOWN5%']

df['updown%'] = np.select(conditions, results, default='STEADY5%')
df.groupby(['date', 'updown%']).agg(
    num=('symbol', 'nunique')).reset_index().head()


,date,updown%,num
0,20120104,DOWN5%,277
1,20120104,STEADY5%,2026
2,20120104,UP5%,17
3,20120105,DOWN5%,885
4,20120105,STEADY5%,1425


### 8. 每天涨幅前10的股票的总成交额和跌幅前10的股票的总成交额比例是多少？

In [48]:
df.sort_values(['date','change%'], ascending=True,inplace=True)

df.groupby('date').agg(down_top10=('amount', lambda x: sum(x[:10])),
                        up_top10=('amount', lambda x: sum(x[-10:]))).assign().assign(ratio=lambda x: x.up_top10 / x.down_top10).head()

,down_top10,up_top10,ratio
date,,,
20120104,456926638.10,660522056.10,1.45
20120105,383022701.40,660035624.90,1.72
20120106,703279193.90,947170129.80,1.35
20120109,558059839.50,1586198245.50,2.84
20120110,2948836472.30,1658431604.10,0.56


### 9. 每天开盘涨停的股票与收盘涨停的股票各有多少？（涨停按照收益率超过1.5%的标准计算）

In [50]:
# 9
df['ret_open'] = df['open']/df['pre_close'] > 1.015
df['ret_close'] = df['close']/df['pre_close'] > 1.015
df.groupby('date')[['ret_open', 'ret_close']].sum().reset_index().head()


,date,ret_open,ret_close
0,20120104,325,70
1,20120105,27,60
2,20120106,56,743
3,20120109,73,2142
4,20120110,95,2125


### 10. 每天统计最近3天出现过开盘涨停、跌停的股票各有多少只？

In [93]:
# use flag created in Q9

# get # of stocks per day
df1=df.groupby('date')[['ret_open','ret_close']].agg('sum').reset_index()
# rolling at 4 days window
df1[['n_openlimit_3d','n_closelimit_3d']] = df1[['ret_open','ret_close']].rolling(4).apply(lambda x: sum(x[:3]))
df1.head()

,date,ret_open,ret_close,n_openlimit_3d,n_closelimit_3d
0,20120104,325,70,NaN,NaN
1,20120105,27,60,NaN,NaN
2,20120106,56,743,NaN,NaN
3,20120109,73,2142,408.00,873.00
4,20120110,95,2125,156.00,2945.00


### 11. 股票每天的成交额变化率和收益率的相关性如何？
### 12. 每天每个行业的总成交额变化率和行业收益率的相关性如何？
### 13. 每天市场的总成交额变化率和市场收益率相关性如何？
### 14. 每天市场的总成交额的变化率和所有股票收益率的标准差相关性如何？
### 15. 每天每个行业的总成交额变化率和行业内股票收益率的标准差相关性如何？
### 16. 上证50、沪深300、中证500指数成分股中，沪股和深股各有多少？
### 17. 上证50、沪深300、中证500指数成分股中，行业分布如何？
### 18. 每天上证50、沪深300、中证500指数成分股的总成交额各是多少？
### 19. 上证50、沪深300、中证500指数日收益率的历史波动率是多少？
### 20. 上证50、沪深300、中证500指数日收益率的相关系数矩阵？
### 21. 上证50、沪深300、去除上证50的沪深300指数日收益率的相关系数矩阵？
### 22. 每天沪深300指数成分占比最大的10只股票是哪些？
### 23. 各个行业的平均每日股票数量从大到小排序是什么？
### 24. 每个行业每天成交额最大的一只股票代码是什么？
### 25. 每个行业每天最大成交额是最小成交额的几倍？
### 26. 每个行业每天成交额最大的5只股票和成交额总和是多少？
### 27. 每个行业每天成交额超过该行业中股票成交额80%分位数的股票的平均收益率是多少？
### 28. 每天成交额最大的10%的股票的平均收益率和成交额最小的10%的股票的平均收益率的相关系数是多少？
### 29. 每天哪些行业的平均成交额高于全市场平均成交额？
### 30. 每天每个股票对市场的超额收益率是多少？
### 31. 每天每个股票对市场去除自身的超额收益率是多少？
### 32. 每天每个股票对行业的超额收益率是多少？
### 33. 每天每个股票对行业去除自身的超额收益率是多少？
### 34. 每个股票每天对市场的超额收益率与对行业的超额收益率的相关系数如何？
### 35. 每天有哪些行业的平均收益率超过市场平均收益率？
### 36. 每天每个行业对市场的超额收益率是多少？
### 37. 每天每个行业对去除本行业后的市场超额收益是多少？
### 38. 每天分别有多少股票是最近连续3个交易日上涨、下跌的？
### 39. 每天分别有多少股票是最近连续3个交易日收益率超过当天市场平均收益率？
### 40. 每天分别有多少股票是最新5个交易日中至少有4个交易日的收益率超过当天市场平均收益率？
### 41. 每个月中，个股月收益超过市场月收益1倍以上的股票有哪些？
### 42. 每个月中，个股月收益超过行业月收益1倍以上的股票有哪些？
### 43. 每个股票的收益率对市场收益率的相关系数最高的10个股票是哪些？
### 44. 每个行业日收益率的历史波动率是多少？（用日收益率计算标准差）
### 45. 各个行业的日收益率的相关系数矩阵如何？哪两个行业相关性最高、最低？
### 46. 各个行业的收益率对市场收益率的相关系数由高到低排列如何？
### 47. 每个月总成交额比上个月下降幅度最大的行业是哪个？
### 48. 数据当中各个股票的最大回撤幅度是多少？（最大回撤是从一个高点到低点的降幅的最大值）
### 49. 每只股票的胜率是多少？（胜率是每天收益率为正数的概率）
### 50. 每只股票的盈亏比是多少？（盈亏比是正收益之和与负收益之和的比值的绝对值）
### 51. 市场的胜率是多少？（市场收益率为正的概率）
### 52. 市场的盈亏比是多少？（市场中每个股票的市值加权正收益和市值加权负收益之比）
### 53. 每个行业的胜率是多少？
### 54. 每个行业的盈亏比是多少？（行业盈亏比是行业内每个股票的市值加权的正收益率和市值加权的负收益率之比）
### 55. 是否存在股票的月成交额超过所在行业当月中某天一天总成交额的情况？
### 56. 每天每个行业编入、编出的股票各有多少？
### 57. 每天每个行业内股票收益率的标准差是多少？
### 58. 每天每个行业内股票收益率的标准差的相关性如何？
### 59. 每天计算出成交额的 z-score （减去均值除以标准差）, 该指标能解释下一天个股超额收益率的多少比例？
### 60. 每个股票的收益率和300、500指数收益率可以回归出一个截距项和2个beta，这两个beta的分布如何？
### 61. 每天开盘后到最高价涨幅最大的100只股票同样也是全天(昨收到今收)涨幅最大的100只股票的比例是多少?
### 62. 每天计算最近三天每天对市场的超额收益率都排进当天前100的股票有哪些?
### 63. 每天计算最近三天每天对行业的超额收益率都排进当天行业前30%的股票有哪些?